In [30]:
# Common imports
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [31]:
# Import from utils
import sys
import importlib
sys.path.append('../')

import utils
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\USUARIO\\Desktop\\DevProjects\\4Geeks_temporal_series\\src\\aquifer\\..\\utils.py'>

# Combinación de datasets para el modelo

In [32]:
aquifer_auser = pd.read_csv('../../data/processed/aquifer_auser.csv')
aquifer_doganella = pd.read_csv('../../data/processed/aquifer_doganella.csv')
aquifer_luco = pd.read_csv('../../data/processed/aquifer_luco.csv')
aquifer_petrignano = pd.read_csv('../../data/processed/aquifer_Doganella.csv')

aquifer_df = pd.concat([aquifer_auser, aquifer_doganella, aquifer_luco, aquifer_petrignano], axis=0)
aquifer_auser.head()

,Date,Rainfall_Gallicano,Rainfall_Pontetetto,Rainfall_Monte_Serra,Rainfall_Orentano,Rainfall_Borgo_a_Mozzano,Rainfall_Piaggione,Rainfall_Calavorno,Rainfall_Croce_Arcana,Rainfall_Tereglio_Coreglia_Antelminelli,...,Temperature_Monte_Serra,Temperature_Ponte_a_Moriano,Temperature_Lucca_Orto_Botanico,Volume_POL,Volume_CC1,Volume_CC2,Volume_CSA,Volume_CSAL,Hydrometry_Monte_S_Quirico,Hydrometry_Piaggione
0,1998-03-05,0.116688,0.177353,0.140604,0.397059,0.166478,0.25,0.149282,0.0,0.076087,...,0.210174,0.084821,0.096070,0.294292,0.026408,0.070559,1.0,1.0,0.290323,0.52459
1,1998-03-06,0.116688,0.177353,0.140604,0.397059,0.166478,0.25,0.149282,0.0,0.076087,...,0.210174,0.084821,0.388646,0.294292,0.026408,0.070559,1.0,1.0,0.290323,0.52459
2,1998-03-07,0.116688,0.177353,0.140604,0.397059,0.166478,0.25,0.149282,0.0,0.076087,...,0.456493,0.084821,0.387191,0.294292,0.026408,0.070559,1.0,1.0,0.290323,0.52459
3,1998-03-08,0.116688,0.177353,0.140604,0.397059,0.166478,0.25,0.149282,0.0,0.076087,...,0.515395,0.084821,0.499272,0.294292,0.026408,0.070559,1.0,1.0,0.290323,0.52459
4,1998-03-09,0.116688,0.177353,0.140604,0.397059,0.166478,0.25,0.149282,0.0,0.076087,...,0.515395,0.084821,0.470160,0.294292,0.026408,0.070559,1.0,1.0,0.290323,0.52459


In [33]:
aquifer_df['Date'] = pd.to_datetime(aquifer_df['Date'], format='%Y-%m-%d')
aquifer_df = aquifer_df.sort_values(by='Date')

In [34]:
utils.get_null_percentage_per_column(aquifer_df)

Rainfall_Simignano                72.964287
Rainfall_Mensano                  72.964287
Rainfall_Siena_Poggio_al_Vento    72.964287
Rainfall_Sovicille                72.964287
Rainfall_Monticiano_la_Pineta     72.964287
                                    ...    
Volume_Pozzo_3                    29.444264
Depth_to_Groundwater_Pozzo_3      29.444264
Volume_Pozzo_4                    29.444264
Volume_Pozzo_1                    29.444264
Date                               0.000000
Length: 63, dtype: float64